# FMLIKH Model: Stress Relaxation

## Learning Objectives

1. Generate **synthetic relaxation data** from NB07 calibrated parameters
2. Analyze **multi-exponential decay** with fractional memory
3. Compare Prony-series-like behavior with power-law modifications
4. Fit FMLIKH to relaxation data

## Prerequisites

- NB07: FMLIKH Flow Curve (calibrated parameters)
- NB03: FIKH Stress Relaxation (single-mode concepts)

## 1. Setup

In [ ]:
import sys
IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"

In [ ]:
%matplotlib inline
import os, sys, time, warnings
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.fikh import FMLIKH

# Robust path resolution for execution from any directory
from pathlib import Path
_nb_dir = Path(__file__).parent if "__file__" in dir() else Path.cwd()
_utils_candidates = [_nb_dir / ".." / "utils", Path("examples/utils"), _nb_dir.parent / "utils"]
for _p in _utils_candidates:
    if (_p / "fikh_tutorial_utils.py").exists():
        sys.path.insert(0, str(_p.resolve()))
        break
from fikh_tutorial_utils import (
    load_fikh_parameters, generate_synthetic_relaxation, save_fikh_results,
    set_model_parameters, print_convergence_summary, compute_fit_quality,
    get_fmlikh_param_names,
)

jax, jnp = safe_import_jax()
verify_float64()
warnings.filterwarnings("ignore", category=FutureWarning)

## 2. Load Calibrated Parameters

In [ ]:
N_MODES = 3
model = FMLIKH(n_modes=N_MODES, include_thermal=False, shared_alpha=True, alpha_structure=0.7)

try:
    params = load_fikh_parameters("fmlikh", "flow_curve")
    set_model_parameters(model, params)
    print("Loaded parameters from NB07")
except FileNotFoundError:
    print("Using default parameters")

## 3. Generate Synthetic Data

In [ ]:
SIGMA_0 = 100.0
time_data, stress_data = generate_synthetic_relaxation(
    model, sigma_0=SIGMA_0, t_end=100.0, n_points=200, noise_level=0.03, seed=42
)

fig, ax = plt.subplots(figsize=(10, 6))
ax.loglog(time_data, stress_data, "ko", markersize=5)
ax.set_xlabel("Time [s]")
ax.set_ylabel("Stress [Pa]")
ax.set_title("Synthetic FMLIKH Relaxation Data")
ax.grid(True, alpha=0.3, which="both")
plt.tight_layout()
display(fig)
plt.close(fig)

## 4. NLSQ Fitting

In [ ]:
# For relaxation: constant strain γ₀ = σ₀/G_total
try:
    G_total = sum(model.parameters.get_value(f'G_{i}') for i in range(N_MODES))
except (KeyError, AttributeError):
    G_total = 1000.0

gamma_0 = SIGMA_0 / max(G_total, 1.0)
strain_data = np.full_like(time_data, gamma_0)

model_fit = FMLIKH(n_modes=N_MODES, include_thermal=False, shared_alpha=True, alpha_structure=0.5)

t0 = time.time()
model_fit.fit(time_data, stress_data, test_mode="relaxation", strain=strain_data, method='scipy')
print(f"NLSQ time: {time.time() - t0:.2f} s")

stress_pred = model_fit.predict(time_data, test_mode="relaxation", strain=strain_data)
metrics = compute_fit_quality(stress_data, stress_pred)
print(f"R² = {metrics['R2']:.6f}")

In [ ]:
time_fine = np.logspace(np.log10(time_data.min()), np.log10(time_data.max()), 300)
strain_fine = np.full_like(time_fine, gamma_0)
stress_pred_fine = model_fit.predict(time_fine, test_mode="relaxation", strain=strain_fine)

fig, ax = plt.subplots(figsize=(10, 6))
ax.loglog(time_data, stress_data, "ko", markersize=5, label="Synthetic data")
ax.loglog(time_fine, stress_pred_fine, "-", lw=2.5, color="C0", label="FMLIKH fit")
ax.set_xlabel("Time [s]")
ax.set_ylabel("Stress [Pa]")
ax.set_title(f"FMLIKH Relaxation Fit (R² = {metrics['R2']:.5f})")
ax.legend()
ax.grid(True, alpha=0.3, which="both")
plt.tight_layout()
display(fig)
plt.close(fig)

## 5. Bayesian Inference

In [ ]:
param_names = get_fmlikh_param_names(n_modes=N_MODES, shared_alpha=True)
initial_values = {n: model_fit.parameters.get_value(n) for n in param_names if n in model_fit.parameters}

NUM_WARMUP, NUM_SAMPLES, NUM_CHAINS = 200, 500, 1

t0 = time.time()
result = model_fit.fit_bayesian(
    time_data, stress_data, test_mode="relaxation", strain=strain_data,
    num_warmup=NUM_WARMUP, num_samples=NUM_SAMPLES, num_chains=NUM_CHAINS,
    initial_values=initial_values, seed=42,
)
print(f"Bayesian time: {time.time() - t0:.1f} s")

In [ ]:
key_params = ["G_0", "G_1", "alpha_structure"]
print_convergence_summary(result, key_params)

## 6. Save Results

In [ ]:
save_fikh_results(model_fit, result, "fmlikh", "relaxation", param_names)

## Key Takeaways

1. **Multi-mode relaxation** shows multiple decay time scales
2. **Fractional order** modifies each mode's decay character
3. **Log-log plots** reveal power-law tails from fractional memory
4. **Relaxation data constrains** mode distribution and alpha